In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re
import string

Datasets loading
---------

In [ ]:
RES_DIR = "../input/"
def load_train_data():
    categories = ['cooking', 'robotics', 'travel', 'crypto', 'diy', 'biology']
    train_data = []
    for cat in categories:
        data = pd.read_csv("{}{}.csv".format(RES_DIR, cat), usecols=['id', 'title','content' ,'tags'])
        data['category'] = cat
        train_data.append(data)
    return pd.concat(train_data)
train_data = load_train_data()
#import the test data

print(train_data.head())

#import test data in same format
test = pd.read_csv("../input/test.csv")
test['tags'] = ''
test['category'] = 'physics'
print(test.head())

#looks fine for me we assume that the corpus is a physics corpus.


**How to find the Physic stopwords** 
---------


make use of the scifit functions: vectorizing > normalizing > multinomila selection

In [ ]:
#define scikit pipeline
# vectorizing document > tfidf normalizing > multinomial selection
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

In [ ]:
#learn words from training set
text_clf = text_clf.fit(train_data['title'],train_data['category'])
#use trained words as stop words
stop_words = stopwords.words('english')
#print(stop_words)
predicted=text_clf.predict(test['title'])

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_data['title'])
X_train_counts.shape

# sparse matrix = X_train_counts
#word index...
print(count_vect.vocabulary_)
      
#Find list of physics words...and using the trainingkit as stopword vocabulary
vectorizerPHYSICS = TfidfVectorizer(sublinear_tf=True,max_df=0.5,stop_words='englisch')
PHYS=vectorizerPHYSICS.fit_transform(test['title'])
print(vectorizerPHYSICS.vocabulary_)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

#learn words
text_clf = text_clf.fit(train_data['title'],train_data['category'])
#use trained words as stop words
predicted_tit=text_clf.predict(test['title'])
predicted_con=text_clf.predict(test['title']+test['content'])

In [ ]:
#create export file for tags
print(predicted_tit)
print(predicted_con)
right=pd.DataFrame(predicted_con)
right.columns = ['tags']
print(zoek.head())
left=pd.DataFrame(test['id'])

result = pd.concat([left,right], axis=1)
# little stat
groep=result.groupby(by='tags',axis=0).count()
print(groep)

Removing html tags and uris from contents
-----------

In [ ]:
uri_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def stripTagsAndUris(x):
    if x:
        # BeautifulSoup on content
        soup = BeautifulSoup(x, "html.parser")
        # Stripping all <code> tags with their content if any
        if soup.code:
            soup.code.decompose()
        # Get all the text out of the html
        text =  soup.get_text()
        # Returning text stripping out all uris
        return re.sub(uri_re, "", text)
    else:
        return ""

In [ ]:
# This could take a while
print(train_data)

for df in train_data.values():
    df["content"] = df["content"].map(stripTagsAndUris)

In [ ]:
from sklearn.datasets.twenty_newsgroups import fetch_20newsgroups
categories = ['alt.atheism','talk.religion.misc','comp.graphics','sci.space',]
remove = ('headers', 'footers', 'quotes')
data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

Removing punctuation from titles and contents
-----------

In [ ]:
def removePunctuation(x):
    # Lowercasing all words
    x = x.lower()
    # Removing non ASCII chars
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    # Removing (replacing with empty spaces actually) all the punctuations
    return re.sub("["+string.punctuation+"]", " ", x)

In [ ]:
for df in dataframes.values():
    df["title"] = df["title"].map(removePunctuation)
    df["content"] = df["content"].map(removePunctuation)

In [ ]:
print(dataframes["robotics"].iloc[1])

Removing stopwords from titles and contents
-----------

In [ ]:
stops = set(stopwords.words("english"))
def removeStopwords(x):
    # Removing all the stopwords
    filtered_words = [word for word in x.split() if word not in stops]
    return " ".join(filtered_words)

In [ ]:
for df in dataframes.values():
    df["title"] = df["title"].map(removeStopwords)
    df["content"] = df["content"].map(removeStopwords)

In [ ]:
print(dataframes["robotics"].iloc[1])

Splitting tags string in a list of tags
-----------

In [ ]:
print(dataframes["robotics"].iloc[1])

Saving preprocessed dataframes to csv
-----------

In [ ]:
categories = ['robotics', 'comp.graphics',
              'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
              'comp.windows.x', 'misc.forsale', 'rec.autos',
              'rec.motorcycles', 'rec.sport.baseball',
              'rec.sport.hockey', 'sci.crypt', 'sci.electronics',
              'sci.med', 'sci.space', 'soc.religion.christian',
              'talk.politics.guns', 'talk.politics.mideast',
              'talk.politics.misc', 'talk.religion.misc']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from operator import itemgetter
from heapq import nlargest
# each phrase here could be document in your list 
# of documents
my_phrases = df['title']
#print(my_phrases)
#  and you want to find the most similar document
#  to this document             
phrase = ["What spin relates subatomic particles  often hear about subatomic particles having a property called spin  actually relate spinning axis likewould think. Which particles spin spin mean actual spinning motion?"]

# You could do it like this:
vectorizer = TfidfVectorizer(min_df=1.0, stop_words='english')
all_phrases = phrase+my_phrases
my_features = vectorizer.fit_transform(all_phrases)
scores = (my_features[0, :] * my_features[1:, :].T).A[0]
best_score = np.argmax(scores)
answer = my_phrases[best_score]
scoress=scores.sort()
print(vectorizer.stop_words_)
print(scoress[-5])

print(my_features[0, :])